# Task 1

In [1]:
import numpy as np

In [2]:
# Confusion matrices

# Red is classified based on uniform random choice.
red = np.array([
    [0.25, 0.25],
    [0.25, 0.25],
])

blue = np.array([
    [0.60, 0.05],
    [0.20, 0.15]
])

def TP(x):
    return x[0, 0]

def FP(x):
    return x[0, 1]

def FN(x):
    return x[1, 0]

def TN(x):
    return x[1, 1]

In [3]:
# Demographic parity

def prob_positive(x):
    return TP(x) + FP(x)

blue_pos = prob_positive(blue)
red_pos = prob_positive(red)

print("demographic parity of blue vs red:", blue_pos / red_pos)

demographic parity of blue vs red: 1.3


In [4]:
# Equal opportunity

def opportunity(x):
    return TP(x) / (TP(x) + FN(x))

blue_opp = opportunity(blue)
red_opp = opportunity(red)

print("opportunity of blue vs red:", blue_opp / red_opp)

opportunity of blue vs red: 1.4999999999999998


In [5]:
# Predictive rate parity

def positive_predictive_value(x):
    return TP(x) / (TP(x) + FP(x))

def negative_predictive_value(x):
    return TN(x) / (FN(x) + TN(x))

print("positive predictive values of blue vs red:", positive_predictive_value(blue) / positive_predictive_value(red))
print("negative predictive values of blue vs red:", negative_predictive_value(blue) / negative_predictive_value(red))

positive predictive values of blue vs red: 1.846153846153846
negative predictive values of blue vs red: 0.8571428571428572


# Task 2
## Key Results
Not finished on time

## Appendix

In [6]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

In [7]:
column_names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "income"
]

In [8]:
train_path = "../../../../datasets/adult/raw/adult.data"
test_path = "../../../../datasets/adult/raw/adult.test"

train_df = pd.read_csv(train_path, header=None, names=column_names, index_col=False)
test_df = pd.read_csv(test_path, header=None, names=column_names, index_col=False, skiprows=1)

train_df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [9]:
# Preprocessing

train_size = train_df.shape[0]
data = pd.concat([train_df, test_df], copy=True)

X, Y = data.iloc[:, :-1], data.iloc[:, -1]
reals_columns = [col for col in X.columns if X[col].dtype != object]
X = pd.get_dummies(X).astype(np.int32)
Y = Y.apply(lambda s: int(">" in s)).astype(np.int32)
Y.name = "income > 50K"

train_x, train_y = X.iloc[:train_size], Y.iloc[:train_size]
test_x, test_y = X.iloc[train_size:], Y.iloc[train_size:]

assert train_x.shape[0] == train_y.shape[0] == train_df.shape[0]
assert test_x.shape[0] == test_y.shape[0] == test_df.shape[0]
assert train_x.shape[0] !=  test_x.shape[0]

In [10]:
# First model is logistic regression

scaler = StandardScaler().fit(train_x[reals_columns])

def scaled(X):
    X = X.copy()
    X[reals_columns] = scaler.transform(X[reals_columns], copy=True)
    return X
 
lreg = LogisticRegression().fit(scaled(train_x), train_y)

print("train accuracy:", accuracy_score(train_y, lreg.predict(scaled(train_x))))
print("test  accuracy:", accuracy_score(test_y, lreg.predict(scaled(test_x))))

train accuracy: 0.853045053898836
test  accuracy: 0.8530188563356059


In [11]:
# Second model is XGBoost

xgb = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
xgb.fit(train_x, train_y)

print("train accuracy:", accuracy_score(train_y, xgb.predict(train_x)))
print("test  accuracy:", accuracy_score(test_y, xgb.predict(test_x)))

train accuracy: 0.8477933724394214
test  accuracy: 0.8482279958233524


In [12]:
# Gender chosen as protected attribute

male_mask = test_x["sex_ Male"] == 1

lreg_preds = lreg.predict(scaled(test_x))
xgb_preds = xgb.predict(test_x)
targets = test_y

model_preds = [("lreg", lreg_preds), ("xgb", xgb_preds)]

In [13]:
def confusion_matrix(y_hat, y):
    cnt = y_hat.shape[0]
    true_positive = sum((y_hat == 1) & (y == 1)) / cnt
    true_negative = sum((y_hat == 0) & (y == 0)) / cnt
    false_positive = sum((y_hat == 1) & (y == 0)) / cnt
    false_negative = sum((y_hat == 0) & (y == 1)) / cnt

    assert true_positive + true_negative + false_positive + false_negative == 1

    return np.array([
        [true_positive, false_positive],
        [false_negative, true_negative]
    ])

def male_confusion(preds, targets):
    return confusion_matrix(preds[male_mask], targets[male_mask])

def female_confusion(preds, targets):
    return confusion_matrix(preds[~male_mask], targets[~male_mask])

In [14]:
def statistical_parity(model_name, preds):
    print("Statistical parity of male vs female")
    print("for model", model_name, end="")
    statistical_parity = prob_positive(male_confusion(preds, targets)) / prob_positive(female_confusion(preds, targets))
    print(" is", statistical_parity)

for args in model_preds:
    statistical_parity(*args)

Statistical parity of male vs female
for model lreg is 3.3185196856729067
Statistical parity of male vs female
for model xgb is 3.2533588479695017


In [15]:
def equal_opportunity(model_name, preds):
    print("Equal opportunity of male vs female")
    print("for model", model_name, end="")
    male_opp = opportunity(male_confusion(preds, targets))
    female_opp = opportunity(female_confusion(preds, targets))
    print(" is", male_opp / female_opp)

for args in model_preds:
    equal_opportunity(*args)

Equal opportunity of male vs female
for model lreg is 1.164381390187842
Equal opportunity of male vs female
for model xgb is 1.1471995362840435


In [16]:
def predictive_parity(model_name, preds):
    print("Predictive parity of male vs female")

    print("for model", model_name, end="")
    male_positive = positive_predictive_value(male_confusion(preds, targets))
    male_negative = negative_predictive_value(male_confusion(preds, targets))
    female_positive = positive_predictive_value(female_confusion(preds, targets))
    female_negative = negative_predictive_value(female_confusion(preds, targets))

    positive_parity = male_positive / female_positive
    negative_parity = male_negative / female_negative
    
    print(" is:", "positive:", positive_parity, "negative:", negative_parity)

for args in model_preds:
    predictive_parity(*args)

Predictive parity of male vs female
for model lreg is: positive: 0.9665689149560117 negative: 0.8943476221577693
Predictive parity of male vs female
for model xgb is: positive: 0.9713795594077285 negative: 0.8806545446700489
